<a href="https://colab.research.google.com/github/dutta-arka/Resonance-Capture-of-Planets/blob/main/Codes/RTP%3B%20Missing_Planets%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import all the required packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

In [2]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv("E2.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1102 entries, 0 to 1101
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   2MASS J19383260+4603591 b  1102 non-null   object 
 1   2MASS J19383260+4603591    1102 non-null   object 
 2   406                        1100 non-null   float64
dtypes: float64(1), object(2)
memory usage: 26.0+ KB


In [3]:
# Group the DataFrame by the names in column 2
grouped = df.groupby(df.iloc[:, 1])

# Initialize an empty list to store dictionaries
output = []

# Initialize a counter
counter = 1

# Iterate over each group
for name, group in grouped:
    # Create a dictionary for the current group
    d = {}
    for index, row in group.iterrows():
        # Extract the key from column 1 and value from column 3
        key = row.iloc[0][-1]  # Extract the last character from the string in column 1
        value = row.iloc[2]
        # Check if the value is not NaN
        if not np.isnan(value):
            d[key] = value
    # Append the dictionary to the output list
    if d:
      output.append({'No.': counter, 'Host Names': name, 'Orbital Period Values': d})
      counter += 1

# Convert the list of dictionaries to a DataFrame
odf = pd.DataFrame(output)

# Print the output as a table
print(odf.to_string(index=False))

 No.              Host Names                                                                                                      Orbital Period Values
   1 2MASS J19383260+4603591                                                                                                   {'c': 1460.0, 'd': 43.0}
   2                  47 UMa                                                                                   {'b': 1078.0, 'c': 2391.0, 'd': 14002.0}
   3                  55 Cnc                                                     {'b': 14.6516, 'c': 44.3989, 'd': 5574.2, 'e': 0.7365474, 'f': 259.88}
   4                  61 Vir                                                                                     {'b': 4.215, 'c': 38.021, 'd': 123.01}
   5                  AU Mic                                                                            {'b': 8.4629991, 'c': 18.858991, 'd': 12.73596}
   6                 CoRoT-7                                                            

In [4]:
results = []
err = 0.05

for index, row in odf.iterrows():
    host_name = row['Host Names']
    orbital_period_values = row['Orbital Period Values']

    ratios = set()  # Use a set to avoid duplicates
    for i in range(len(orbital_period_values)):
        for j in range(i + 1, len(orbital_period_values)):
            for denominator in range(1, 20):
                try:
                    ratio = orbital_period_values[list(orbital_period_values.keys())[j]] / (
                        denominator * orbital_period_values[list(orbital_period_values.keys())[i]]
                    )
                    # Check if the ratio is close to an integer
                    rounded_ratio = round(ratio)
                    if abs(ratio - rounded_ratio) <= err and (rounded_ratio != 0 and denominator != 0):
                        # Simplify the ratio if possible
                        gcd = abs(math.gcd(int(rounded_ratio), denominator))

                        # Ensure a > b in the simplified ratio
                        numerator = int(rounded_ratio // gcd)
                        denominator = denominator // gcd
                        if numerator < denominator:
                            numerator, denominator = denominator, numerator

                        simplified_ratio = f"{numerator}:{denominator}"
                        ratios.add(simplified_ratio)
                except ZeroDivisionError:
                    pass  # Handle potential zero division errors

    if ratios:
        results.append({'No.': index + 1, 'Host Names': host_name, 'Orbital Period Values': orbital_period_values, 'Ratios': list(ratios)})

# Create a DataFrame from the results
rdf = pd.DataFrame(results)

print(rdf.to_string(index=False))

 No.     Host Names                                                                                                      Orbital Period Values                                                                                                         Ratios
   2         47 UMa                                                                                   {'b': 1078.0, 'c': 2391.0, 'd': 14002.0}                                                                                               [3:2, 13:1, 6:1]
   3         55 Cnc                                                     {'b': 14.6516, 'c': 44.3989, 'd': 5574.2, 'e': 0.7365474, 'f': 259.88}                                                 [18:7, 18:1, 3:2, 20:19, 2:1, 3:1, 9:2, 14:9, 6:1, 17:1, 19:5]
   4         61 Vir                                                                                     {'b': 4.215, 'c': 38.021, 'd': 123.01}                                                                                                

In [5]:
import pandas as pd

# Function to convert a ratio to a fraction and check if both numerator and denominator are less than 3.5
def check_ratio_validity(ratio):
    num, denom = map(float, ratio.split(':'))
    fraction = num / denom
    if fraction <= 3.5:
        return num and denom
    else:
        return None

# Flatten the list of ratios and filter out ratios with values less than 3.5
valid_ratios = [ratio for sublist in rdf['Ratios'] for ratio in sublist if check_ratio_validity(ratio)]

# Count occurrences of each valid ratio
ratio_counts = pd.Series(valid_ratios).value_counts()

# Calculate percentage of each ratio
ratio_percentages = (ratio_counts / ratio_counts.sum()) * 100

# Sort ratios based on occurrences in descending order
sorted_ratios = ratio_counts.index.tolist()

# Select top 5 ratios and their percentages
top_5_ratios = sorted_ratios[:5]
top_5_percentages = ratio_percentages[top_5_ratios]

# Create a DataFrame for top 5 ratios and percentages
top_5_df = pd.DataFrame({'Ratio': top_5_ratios, 'Percentage': top_5_percentages})
top_5_df.reset_index(drop=True, inplace=True)

print("Top 5 most likely ratios and their percentages:")
print(top_5_df)

Top 5 most likely ratios and their percentages:
  Ratio  Percentage
0   2:1   29.230769
1   3:1   22.461538
2   1:1   11.384615
3   3:2    7.692308
4   5:2    3.692308


In [9]:
# Create an empty list to store the results
missing_planets_data = []

# Iterate over each row in the DataFrame
for index, row in rdf.iterrows():
    host_name = row['Host Names']
    orbital_period_values = row['Orbital Period Values']
    ratios_present = row['Ratios']

    # Convert the orbital period values to a list of tuples (planet_name, orbital_period)
    periods = list(orbital_period_values.items())

    # Extract the planets with their orbital periods
    planets_with_periods = [(planet_name, period) for planet_name, period in periods]

    # Calculate the expected orbital periods for missing ratios
    for missing_ratio in set(['2:1', '3:1', '1:1', '3:2', '5:2']) - set(ratios_present):
        # Extract the numerator and denominator from the missing ratio
        numerator, denominator = map(int, missing_ratio.split(':'))

        # Iterate over the known planets and their orbital periods
        for planet_name, period in planets_with_periods:
            # Calculate the expected orbital period based on the missing ratio
            expected_periods = [period * k / numerator for k in range(1, denominator + 1)]

            # Add the missing planets to the results
            for i, expected_period in enumerate(expected_periods):
                missing_planets_data.append({'Host Name': host_name,
                                             'Possible Missing Planets': f"{expected_period:.6f} +/- {period * err:.6f}",
                                             'Ratio': f"{numerator}:{denominator}"})

# Create a DataFrame from the list of results
missing_planets_df = pd.DataFrame(missing_planets_data)

# Save the DataFrame as a CSV file
missing_planets_df.to_csv('missing_planets.csv', index=False)

# Print the DataFrame
print(missing_planets_df)

     Host Name    Possible Missing Planets Ratio
0       47 UMa    539.000000 +/- 53.900000   2:1
1       47 UMa  1195.500000 +/- 119.550000   2:1
2       47 UMa  7001.000000 +/- 700.100000   2:1
3       47 UMa    359.333333 +/- 53.900000   3:1
4       47 UMa   797.000000 +/- 119.550000   3:1
...        ...                         ...   ...
4708   ups And       1.846813 +/- 0.230852   5:2
4709   ups And     48.251600 +/- 12.062900   5:2
4710   ups And     96.503200 +/- 12.062900   5:2
4711   ups And    255.292000 +/- 63.823000   5:2
4712   ups And    510.584000 +/- 63.823000   5:2

[4713 rows x 3 columns]
